In [ ]:
als nächste: die jahre berücktrichten!!!!!

In [11]:
import os
import pandas as pd
import json
import requests
from meteostat import Daily, Stations
from datetime import datetime
from data_preparation import download_files, files, output_folder
from data_preparation import data_import, data_cleaning, weather_data, geo_data


In [19]:
download_files(files, output_folder)

Downloaded: waqi-covid-2025.csv
Downloaded: waqi-covid-2015H1.csv
Downloaded: waqi-covid-2016H1.csv
Downloaded: waqi-covid-2017H1.csv
Downloaded: waqi-covid-2018H1.csv
Downloaded: waqi-covid-2019Q1.csv
Downloaded: waqi-covid-2019Q2.csv
Downloaded: waqi-covid-2019Q3.csv
Downloaded: waqi-covid-2019Q4.csv
Downloaded: waqi-covid-2020Q1.csv
Downloaded: waqi-covid-2020Q2.csv
Downloaded: waqi-covid-2020Q3.csv
Downloaded: waqi-covid-2020Q4.csv
Downloaded: waqi-covid-2021Q1.csv
Downloaded: waqi-covid-2021Q2.csv
Downloaded: waqi-covid-2021Q3.csv
Downloaded: waqi-covid-2021Q4.csv
Downloaded: waqi-covid-2022Q1.csv
Downloaded: waqi-covid-2022Q2.csv
Downloaded: waqi-covid-2022Q3.csv
Downloaded: waqi-covid-2022Q4.csv
Downloaded: waqi-covid-2023Q1.csv
Downloaded: waqi-covid-2023Q2.csv
Downloaded: waqi-covid-2023Q3.csv
Downloaded: waqi-covid-2023Q4.csv
Downloaded: airquality-covid19-cities.json
Downloaded: unsd-citypopulation-year-both.csv


In [12]:
data_import()   

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.


,Date,Country,City,Specie,count,min,max,median,variance
0,2015-01-06,KR,Jeonju,co,124,0.1,12.3,4.5,55.74
1,2015-01-22,KR,Jeonju,co,116,4.5,10.0,6.7,16.09
2,2015-03-30,KR,Jeonju,co,118,1.2,11.2,5.6,35.98
3,2015-05-27,KR,Jeonju,co,93,2.3,5.6,3.4,6.54
4,2015-02-03,KR,Jeonju,co,133,4.5,13.4,7.8,39.24
...,...,...,...,...,...,...,...,...,...
14257913,2024-10-05,MN,Ulan Bator,pressure,46,1027.0,1031.0,1029.0,15.54
14257914,2025-02-17,MN,Ulan Bator,pressure,264,1025.0,1027.0,1026.0,6.06
14257915,2025-02-26,MN,Ulan Bator,pressure,264,1002.0,1016.0,1009.0,213.15
14257916,2024-04-04,MN,Ulan Bator,pressure,240,1016.0,1019.0,1018.0,6.68


In [ ]:
df_population = pd.read_csv(
    '/Users/whypk/01Projekte/air_quality/data/flat-ui__data-Tue Mar 04 2025.csv',
    sep=',', 
    header=0,
    engine='python',  # Hilft oft bei Parsing-Problemen
    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
    encoding='utf-8'  # Falls Sonderzeichen vorhanden sind
)

In [23]:
print(df_population.sample(5).T)
print(df_population.columns)
print(df_population.dtypes)
print(df_population.shape)

                                   6228  ...                    6535
Country or Area              Costa Rica  ...                    Cuba
Year                               2019  ...                    2011
Area                              Total  ...                   Total
Sex                          Both Sexes  ...              Both Sexes
City                       Desamparados  ...              Guantánamo
City type                   City proper  ...             City proper
Record Type          Estimate - de jure  ...      Estimate - de jure
Reliability      Final figure, complete  ...  Final figure, complete
Source Year                      2019.0  ...                  2013.0
Value                          242983.0  ...           207739.236485
Value Footnotes                      74  ...                     NaN

[11 rows x 5 columns]
Index(['Country or Area', 'Year', 'Area', 'Sex', 'City', 'City type',
       'Record Type', 'Reliability', 'Source Year', 'Value',
       'Value Foot

In [ ]:
df_population.columns = ['City', 'population']
df_population.dropna(inplace=True)
df_population['population'] = df_population['population'].astype(int)
df_population["City"] = df_population["City"].str.lower().str.strip()

In [ ]:
def population(df):
    '''
    Fügt dem jeder Stadt Einwohener hinzu
    '''
    df = df_copy()
    
    df_population = pd.read_csv(
                    './data/flat-ui__data-Tue Mar 04 2025.csv',
                    sep=',', 
                    header=0,
                    engine='python',  # Hilft oft bei Parsing-Problemen
                    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
                    encoding='utf-8')  # Falls Sonderzeichen vorhanden sind

    df_population = df_population[['City', 'Value']]

    output_path = './data/population_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df_population.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    # Merge der beiden DataFrames basierend auf der "City"-Spalte
    df = pd.merge(df, df_population, on='City', how='left')

    return(df)

In [ ]:
output_path = './data/population_data.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_population.to_csv(output_path, index=False)
print(f"✅ Datei wurde gespeichert: {output_path}")

✅ Datei wurde gespeichert: ./data/population_data.csv


In [41]:
# Merge der beiden DataFrames basierend auf der "City"-Spalte
df_merged = pd.merge(df, df_population, on='City', how='left')

NameError: name 'df' is not defined

In [33]:
# Funktion für Übersicht über dtypes, missing values, unique values etc.
def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))
overview(df_population)

,dtype,total,missing,missing%,n_uniques,uniques%,uniques
City,object,29857,254,0.843546,5156,17.123310,"[Durrës, TIRANA, Adrar, Ain Defla, Ain Temouch..."
population,float64,29857,254,0.843546,27999,92.985952,"[113249.0, 418495.0, 392863.0, 200834.0, 45028..."


In [25]:
df_filtered_population = df_population[df_population['City'].isin(df['City'])][['Country or Area','City', 'Year', 'City type', 'Value']]
df_filtered_population['Value'] = df_filtered_population['Value'].astype(int)
df_filtered_population.head()

NameError: name 'df' is not defined